In [ ]:
try:
  # This %tensorflow_version magic only works in Colab.
  %tensorflow_version 1.x
except Exception:
  pass
# For your non-Colab code, be sure you have tensorflow==1.15
import tensorflow as tf

tf.enable_eager_execution()

import os
import numpy as np
import matplotlib.pyplot as plt
from keras import backend as K

In [ ]:
IMAGE_SIZE = 96
REPDATA_PATH = '../data/input/Training/*/*'
KERAS_MODEL = 'builtin_mobilenetv2-longrun.h5'

In [ ]:
# A generator that provides a representative dataset
def representative_data_gen():
    dataset_list = tf.data.Dataset.list_files(REPDATA_PATH)
    for i in range(1000):
        image = next(iter(dataset_list))
        image = tf.io.read_file(image)
        image = tf.io.decode_jpeg(image, channels=1)
        image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
        image = tf.cast(image, tf.float32)
        image = tf.cast(image / 255., tf.float32)
        image = tf.expand_dims(image, 0)
        yield [image]

converter = tf.lite.TFLiteConverter.from_keras_model_file(KERAS_MODEL)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()

with open('builtin_mobilenetv2-longrun.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
!edgetpu_compiler builtin_mobilenetv2-longrun.tflite